The main topic of these Notebook is single object detection, which means getting a model to draw a box around every key object in an image, and label each one correctly.  we can use transfer learning from an <b>ImageNet</b> classifier that was never even trained to do detection.We are going to follow these steps:
- Classify the biggest object
- Predict Bounding box of biggest object
- Put all together to predict and classify the biggest object

we’ll use a single model to do both these at the same time.

###upgrade and import some packages

In [1]:
! pip install --upgrade git+https://github.com/keras-team/keras.git \
                        git+https://github.com/valeoai/dl_utils.git \
                        imageio

In [7]:
import keras
from keras.models import Sequential, Model 
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Dropout, Flatten,Dense
from tensorflow.keras.optimizers import Adam

import numpy as np
import os
from matplotlib import image,patches,patheffects
import matplotlib.pyplot as plt
from PIL import Image



## Pascal VOC
We will be looking at the [Pascal VOC](http://host.robots.ox.ac.uk/pascal/VOC/) dataset. It's quite slow, so you may prefer to download from [this mirror](https://pjreddie.com/projects/pascal-voc-dataset-mirror/). There are two different competition/research datasets, from 2007 and 2012. We'll be using the 2007 version. later we can use the larger 2012 for better results.



import os
if not os.path.isdir('./VOCdevkit/'):
    ! wget http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
    ! tar -xf VOCtrainval_06-Nov-2007.tar
    #! wget https://storage.googleapis.com/coco-dataset/external/PASCAL_VOC.zip
    #! unzip -q PASCAL_VOC.zip -d VOCdevkit/VOC2007
    #! rm -Rf VOCdevkit/VOC2007/PASCAL_VOC
    !mv  ../input/pascal-voc-2007/PASCAL_VOC/PASCAL_VOC/pascal_train2012.json VOCdevkit/VOC2007/pascal_train2007.json
      

In [ ]:
 !mv  ../input/pascal-voc-2007/PASCAL_VOC/PASCAL_VOC/pascal_train2007.json VOCdevkit/VOC2007/pascal_train2007.json

In [2]:
from pathlib import Path

PATH = Path('../input/pascal-voc-2007/PASCAL_VOC/PASCAL_VOC')
for i in PATH.iterdir(): print(i)
    
JPEGS = PATH/'JPEGImages'

so, you have:
- <kbd>JPEGImages</kbd> folder containing all the images
- <kbd>Annotations</kbd> folder containing all the annotation, one XML annotation file per image file

The original version were in XML, which is a little hard to work with nowadays, so we use the more recent JSON version

In [3]:
import json
BD = json.load((PATH/'pascal_train2007.json').open()) # it loads a dictionary
print('the dictionary of keys: ',BD.keys())

In [8]:
# Then, we can acces to the image
BD['images'][:5]


In [7]:
# The annotations
BD['annotations'][:2]

In [9]:
# and all category
BD['categories']

In [10]:
import pandas as pd

In [12]:
df1=pd.DataFrame(BD['images'])
df1.to_csv('images.csv')

In [15]:
df2=pd.DataFrame(BD['annotations'])
df2.to_csv('annotations.csv')

In [17]:
df3=pd.DataFrame(BD['categories'])
df3.to_csv('categories.csv')